## 웹 크롤링

### 인터넷 접속

In [34]:
from urllib.request import urlopen, Request

# 도시별 날씨 검색 함수
def get_weather(city):
    # 기상청 홈페이지 도시별 날씨 페이지
    url = 'https://www.weather.go.kr/w/obs-climate/land/city-obs.do'
    page = urlopen(url=url)

    text = page.read().decode('utf-8')
    text = text[text.find(f'>{city}</a>'):]

    # 기온 가져오기
    for i in range(7):
        text = text[text.find('<td>')+1:]

    start = 3
    end = text.find('</td>')
    current_temp = text[start:end]
    print(f'{city}의 현재 기온은 {current_temp}˚C 입니다.')

    # 습도 가져오기
    for i in range(3):
        text = text[text.find('<td>')+1:]

    start = 3
    end = text.find('</td>')
    current_humid = text[start:end]
    print(f'{city}의 현재 습도는 {current_humid}% 입니다.')
    
get_weather('부산')

부산의 현재 기온은 11.6˚C 입니다.
부산의 현재 습도는 26% 입니다.


### OpneAPI 크롤링

- 공공데이터 포털
    - http://www.data.go.kr
    - 부산광역시 버스정보안내기 현황

In [35]:
import requests
from urllib.parse import quote, unquote,urlencode  # 한글을 URL인코드로 변환해주는 함수
import json
import ssl

# stationName 정류소 이름
def getDataPortalSearch(stationName, type):
    api_url = 'https://apis.data.go.kr/6260000/BusanTblBusinfoeqStusService/getTblBusinfoeqStusInfo'
    queryString = "?" + urlencode(
        {    
            'serviceKey': 'EmCpfwaIYY1h6xUdXPg6c9DxsctH9Bau9ueNoGXoAs0zEckxoWbkI1sdOY/4+p6z3so0n51cffoU0q5ha+hQMQ==',
            'pageNo': '1',
            'numOfRows': '10',
            'resultType': type,
            'stationLoc' : stationName
        }
    )

    total_url = api_url + queryString
    response = requests.get(total_url, verify=False)
    return response.text
        
result = getDataPortalSearch('롯데백화점','json')
json_data = json.loads(result)
station_data = json_data['getTblBusinfoeqStusInfo']['body']['items']['item']

# for i in station_data:
#     print(item)




c:\DEV\Langs\Python311\Lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apis.data.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


### OpenAPI로 가져온 데이터 지도 표시



In [41]:
import folium

# 정류소 중 제일 첫번째 인덱스 정류소 위경도를 중심으로 잡음
if len(station_data) > 0: 
    center_lat = station_data[0]['lat']
    center_lng = station_data[0]['lng']

m = folium.Map(location = [center_lat, center_lng], zoom_start = 13)

# 전체 정류소 위치값을 마커로 표시
for item in station_data:
    folium.Marker(location =[item['lat'], item['lng']], popup=item['stationLoc']).add_to(m)

m